<a href="https://colab.research.google.com/github/vkrisvasan/GeminiAI_KV/blob/main/AgentEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#build and deploy an agent that uses the Gemini Pro model, Python functions as tools, and LangChain for orchestration
#create storage bucket in https://console.cloud.google.com/storage/browser
#create project in

In [1]:
!pip install --upgrade --quiet \
    google-cloud-aiplatform==1.51.0 \
    langchain==0.1.20 \
    langchain-google-vertexai==1.0.3 \
    cloudpickle==3.0.0 \
    pydantic==2.7.1 \
    requests

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [5]:
import os
import getpass
credential_names = ["GCP_PROJECT_ID", "LOCATION", "STAGING_BUCKET"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

In [12]:
PROJECT_ID = os.environ["GCP_PROJECT_ID"]
LOCATION = os.environ["LOCATION"]
STAGING_BUCKET = os.environ["STAGING_BUCKET"]
STAGING_BUCKET_FULL = "gs://{}".format(STAGING_BUCKET)


In [13]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET_FULL)

In [14]:
import requests
from vertexai.preview import reasoning_engines

In [15]:
model = "gemini-1.5-pro-preview-0409"

In [16]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [17]:
get_exchange_rate(currency_from="USD", currency_to="SEK")

{'amount': 1.0, 'base': 'USD', 'date': '2024-06-12', 'rates': {'SEK': 10.4361}}

In [18]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [19]:
agent.query(input="What's the exchange rate from US dollars to Swedish currency today?")

{'input': "What's the exchange rate from US dollars to Swedish currency today?",
 'output': 'The exchange rate from US dollars to Swedish krona today is 1 USD to 10.4361 SEK. \n',
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'],
    'kwargs': {'tool': 'get_exchange_rate',
     'tool_input': {'currency_from': 'USD', 'currency_to': 'SEK'},
     'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'SEK'}`\n\n\n",
     'type': 'AgentActionMessageLog',
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'additional_kwargs': {'function_call': {'name': 'get_exchange_rate',
          'arguments': '{"currency_from": "USD", "currency_to": "SEK"}'}},
        'response_metadata': {'is_blocked': False,
         'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH',
    

In [20]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

In [ ]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform==1.51.0",
        "langchain==0.1.20",
        "langchain-google-vertexai==1.0.3",
        "cloudpickle==3.0.0",
        "pydantic==2.7.1",
        "requests",
    ],
)

In [ ]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Swedish currency today?"
)